In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
import glob
from scipy.sparse import csr_matrix
import gzip
from tempfile import TemporaryDirectory
import shutil

from scipy import sparse

import matplotlib.pyplot as plt

import mygene

import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
pandas2ri.activate()
# %load_ext rpy2.ipython

import rpy2.rinterface_lib.callbacks
import logging
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

import sys  
sys.path.insert(0, '/lustre/groups/ml01/code/karin.hrovatin/diabetes_analysis/')
import helper as h
from importlib import reload
reload(h)
import helper as h
from constants import SAVE

In [11]:
ro.r('library("scran")')
ro.r('library("BiocParallel")')

array(['BiocParallel', 'scran', 'SingleCellExperiment',
       'SummarizedExperiment', 'Biobase', 'GenomicRanges', 'GenomeInfoDb',
       'IRanges', 'S4Vectors', 'BiocGenerics', 'parallel', 'stats4',
       'MatrixGenerics', 'matrixStats', 'tools', 'stats', 'graphics',
       'grDevices', 'utils', 'datasets', 'methods', 'base'], dtype='<U20')

In [2]:
data_path='/lustre/groups/ml01/workspace/karin.hrovatin/data/pancreas/scRNA/'
path_genes='/lustre/groups/ml01/workspace/karin.hrovatin/data/pancreas/gene_lists/'

## GSE137909 - STRT-seq

In [4]:
dataset='GSE137909'
path_ds=data_path+dataset+'/GEO/'

In [6]:
x=pd.read_table(path_ds+'GSE137909_UMI.txt',index_col=0)

In [7]:
x

,Symbol,Ngn3Cre_Control_1_001,Ngn3Cre_Control_1_002,Ngn3Cre_Control_1_003,Ngn3Cre_Control_1_004,Ngn3Cre_Control_1_005,Ngn3Cre_Control_1_006,Ngn3Cre_Control_1_007,Ngn3Cre_Control_1_008,Ngn3Cre_Control_1_009,...,Ngn3Cre_P3_1_195,Ngn3Cre_P3_1_196,Ngn3Cre_P3_1_197,Ngn3Cre_P3_1_198,Ngn3Cre_P3_1_199,Ngn3Cre_P3_1_200,Ngn3Cre_P3_1_201,Ngn3Cre_P3_1_202,Ngn3Cre_P3_1_203,Ngn3Cre_P3_1_204
ID,,,,,,,,,,,,,,,,,,,,,
ENSMUSG00000000001,Gnai3,3,9,3,7,3,2,0,7,5,...,45,33,17,1,0,2,36,14,17,21
ENSMUSG00000000003,Pbsn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSMUSG00000000028,Cdc45,0,0,0,0,0,0,0,0,0,...,18,19,0,0,0,0,0,0,0,0
ENSMUSG00000000031,H19,0,0,0,0,0,0,0,0,0,...,8,0,0,0,0,0,0,0,0,0
ENSMUSG00000000037,Scml2,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSMUSG00000102128,RP24-444C13.15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSMUSG00000102129,RP24-89D11.9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSMUSG00000102130,Olfr1360,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
x_norm=pd.read_table(path_ds+'GSE137909_TM0.1M.txt',index_col=0)

In [9]:
x_norm

,Symbol,Ngn3Cre_Control_1_001,Ngn3Cre_Control_1_002,Ngn3Cre_Control_1_003,Ngn3Cre_Control_1_004,Ngn3Cre_Control_1_005,Ngn3Cre_Control_1_006,Ngn3Cre_Control_1_007,Ngn3Cre_Control_1_008,Ngn3Cre_Control_1_009,...,Ngn3Cre_P3_1_195,Ngn3Cre_P3_1_196,Ngn3Cre_P3_1_197,Ngn3Cre_P3_1_198,Ngn3Cre_P3_1_199,Ngn3Cre_P3_1_200,Ngn3Cre_P3_1_201,Ngn3Cre_P3_1_202,Ngn3Cre_P3_1_203,Ngn3Cre_P3_1_204
ID,,,,,,,,,,,,,,,,,,,,,
ENSMUSG00000000001,Gnai3,2.98,16.62,5.43,20.85,25.59,12.14,0.0,15.66,13.06,...,17.59,46.21,7.36,2.48,0.0,16.08,40.21,13.07,46.85,24.84
ENSMUSG00000000003,Pbsn,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00
ENSMUSG00000000028,Cdc45,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,...,7.04,26.61,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00
ENSMUSG00000000031,H19,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,...,3.13,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00
ENSMUSG00000000037,Scml2,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,...,0.39,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSMUSG00000102128,RP24-444C13.15,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00
ENSMUSG00000102129,RP24-89D11.9,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00
ENSMUSG00000102130,Olfr1360,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00


In [10]:
# Make sure index of x tables matches
if (x.index!=x_norm.index).any() or (x.columns!=x_norm.columns).any():
    raise ValueError('Tables not matching')

In [11]:
# Make expression adata
adata=sc.AnnData(csr_matrix(x.iloc[:,1:].T),var=pd.DataFrame(x.iloc[:,0]),
                obs=pd.DataFrame(index=x.iloc[:,1:].columns))
adata.var.columns=['gene_symbol']
adata.layers['raw']=adata.X.copy()
adata.layers['normalised_original']=csr_matrix(x_norm.iloc[:,1:].T)

Load metadata

In [12]:
obs1=pd.read_table(path_ds+'GSE137909_metadata.txt',index_col=1)

In [13]:
obs1

,Sample name,source name,organism,characteristics: Strain,characteristics: Treatment,characteristics: Insulin implants,characteristics: Time after STZ,characteristics: Age,characteristics: Batch,molecule,description,processed data file,processed data file .1,raw file
title,,,,,,,,,,,,,,
Ngn3Cre_Control_1_001,Sample 0001,Pancreas,Mus musculus,Ngn3-Cre; Rosa-RFP,Control,-,-,Adult,1,mRNA,NaN,UMI.txt.gz,TM0.1M.txt.gz,m180819_06_N06_b001.R1.fq.gz
Ngn3Cre_Control_1_002,Sample 0002,Pancreas,Mus musculus,Ngn3-Cre; Rosa-RFP,Control,-,-,Adult,1,mRNA,NaN,UMI.txt.gz,TM0.1M.txt.gz,m180819_06_N06_b002.R1.fq.gz
Ngn3Cre_Control_1_003,Sample 0003,Pancreas,Mus musculus,Ngn3-Cre; Rosa-RFP,Control,-,-,Adult,1,mRNA,NaN,UMI.txt.gz,TM0.1M.txt.gz,m180819_06_N06_b003.R1.fq.gz
Ngn3Cre_Control_1_004,Sample 0004,Pancreas,Mus musculus,Ngn3-Cre; Rosa-RFP,Control,-,-,Adult,1,mRNA,NaN,UMI.txt.gz,TM0.1M.txt.gz,m180819_06_N06_b004.R1.fq.gz
Ngn3Cre_Control_1_005,Sample 0005,Pancreas,Mus musculus,Ngn3-Cre; Rosa-RFP,Control,-,-,Adult,1,mRNA,NaN,UMI.txt.gz,TM0.1M.txt.gz,m180819_06_N06_b005.R1.fq.gz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ngn3Cre_P12_2_070,Sample 2995,Pancreas,Mus musculus,Ngn3-Cre; Rosa-RFP,-,-,-,P12,2,mRNA,NaN,UMI.txt.gz,TM0.1M.txt.gz,Ngn3Cre_P12_2_070.fq.gz
Ngn3Cre_P12_2_071,Sample 2996,Pancreas,Mus musculus,Ngn3-Cre; Rosa-RFP,-,-,-,P12,2,mRNA,NaN,UMI.txt.gz,TM0.1M.txt.gz,Ngn3Cre_P12_2_071.fq.gz
Ngn3Cre_P12_2_072,Sample 2997,Pancreas,Mus musculus,Ngn3-Cre; Rosa-RFP,-,-,-,P12,2,mRNA,NaN,UMI.txt.gz,TM0.1M.txt.gz,Ngn3Cre_P12_2_072.fq.gz


In [14]:
obs2=pd.read_table(path_ds+'GSE137909_series_matrix.txt',index_col=0,skiprows=48)

In [15]:
obs2

,Ngn3Cre_Control_1,Ngn3Cre_Control_2,Ngn3Cre_Control_3,Ngn3Cre_STZ_D6_1,Ngn3Cre_STZ_D12_1,Ngn3Cre_STZ_D30_1,Ngn3Cre_STZ_D42_1,Ngn3Cre_STZ_M2_1,Ngn3Cre_STZ_M5_1,Ngn3Cre_STZ_M9_1,Ngn3Cre_P3_1,Ngn3Cre_P21_1,Ngn3Cre_STZ_D12_2,Ngn3Cre_P12_1,Ngn3Cre_P12_2,Ngn3Cre_STZ_M2_NoIns_1,Ngn3Cre_STZ_M2_NoIns_2
!Sample_title,,,,,,,,,,,,,,,,,
!Sample_geo_accession,GSM4094193,GSM4094194,GSM4094195,GSM4094197,GSM4094198,GSM4094199,GSM4094200,GSM4094201,GSM4094202,GSM4094203,GSM4273879,GSM4273880,GSM4273881,GSM4405432,GSM4405433,GSM4405434,GSM4405435
!Sample_status,Public on Apr 23 2020,Public on Apr 23 2020,Public on Apr 23 2020,Public on Apr 23 2020,Public on Apr 23 2020,Public on Apr 23 2020,Public on Apr 23 2020,Public on Apr 23 2020,Public on Apr 23 2020,Public on Apr 23 2020,Public on Apr 23 2020,Public on Apr 23 2020,Public on Apr 23 2020,Public on Apr 23 2020,Public on Apr 23 2020,Public on Apr 23 2020,Public on Apr 23 2020
!Sample_submission_date,Sep 24 2019,Sep 24 2019,Sep 24 2019,Sep 24 2019,Sep 24 2019,Sep 24 2019,Sep 24 2019,Sep 24 2019,Sep 24 2019,Sep 24 2019,Jan 15 2020,Jan 15 2020,Jan 15 2020,Mar 10 2020,Mar 10 2020,Mar 10 2020,Mar 10 2020
!Sample_last_update_date,Apr 23 2020,Apr 23 2020,Apr 23 2020,Apr 23 2020,Apr 23 2020,Apr 23 2020,Apr 23 2020,Apr 23 2020,Apr 23 2020,Apr 23 2020,Apr 23 2020,Apr 23 2020,Apr 23 2020,Apr 23 2020,Apr 23 2020,Apr 23 2020,Apr 23 2020
!Sample_type,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA
!Sample_channel_count,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
!Sample_source_name_ch1,Ngn3Cre_Control_pancreas,Ngn3Cre_Control_pancreas,Ngn3Cre_Control_pancreas,Ngn3Cre_STZ_D6_pancreas,Ngn3Cre_STZ_D12_pancreas,Ngn3Cre_STZ_D30_pancreas,Ngn3Cre_STZ_D42_pancreas,Ngn3Cre_STZ_M2_pancreas,Ngn3Cre_STZ_M5_pancreas,Ngn3Cre_STZ_M9_pancreas,Pancreas,Pancreas,Pancreas,Pancreas,Pancreas,Pancreas,Pancreas
!Sample_organism_ch1,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus
!Sample_characteristics_ch1,genotype/variation: Ngn3-Cre; Rosa-RFP,genotype/variation: Ngn3-Cre; Rosa-RFP,genotype/variation: Ngn3-Cre; Rosa-RFP,genotype/variation: Ngn3-Cre; Rosa-RFP,genotype/variation: Ngn3-Cre; Rosa-RFP,genotype/variation: Ngn3-Cre; Rosa-RFP,genotype/variation: Ngn3-Cre; Rosa-RFP,genotype/variation: Ngn3-Cre; Rosa-RFP,genotype/variation: Ngn3-Cre; Rosa-RFP,genotype/variation: Ngn3-Cre; Rosa-RFP,strain: Ngn3-Cre; Rosa-RFP,strain: Ngn3-Cre; Rosa-RFP,strain: Ngn3-Cre; Rosa-RFP,strain: Ngn3-Cre; Rosa-RFP,strain: Ngn3-Cre; Rosa-RFP,strain: Ngn3-Cre; Rosa-RFP,strain: Ngn3-Cre; Rosa-RFP


In [16]:
obs=pd.read_excel(path_ds+'1-s2.0-S2212877820300569-mmc1.xlsx',skiprows=1,index_col=0)

In [17]:
obs

,Strain,Treatment,Insulin implants,Time after STZ,Age at death,Batch,# Raw Read Count,# Mapped Read Count,# Unique Read Count,# Total UMI Count,# Detected Gene Count,Putative Cell Type,Putative β-cell Group
Sample,,,,,,,,,,,,,
Ngn3Cre_Control_1_001,Ngn3-Cre; Rosa-RFP,Control,-,-,2M,1,905227,715470,464307,105381,1909,acinar,-
Ngn3Cre_Control_1_002,Ngn3-Cre; Rosa-RFP,Control,-,-,2M,1,999780,793509,782840,72892,5497,beta,1
Ngn3Cre_Control_1_003,Ngn3-Cre; Rosa-RFP,Control,-,-,2M,1,1081602,845816,837268,74691,5401,beta,1
Ngn3Cre_Control_1_004,Ngn3-Cre; Rosa-RFP,Control,-,-,2M,1,368827,286380,282355,49686,4588,beta,1
Ngn3Cre_Control_1_005,Ngn3-Cre; Rosa-RFP,Control,-,-,2M,1,161912,78510,77434,24195,3370,beta,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ngn3Cre_P21_1_239,Ngn3-Cre; Rosa-RFP,-,-,-,P21,1,3982049,2711270,2671481,211457,6649,beta,1
Ngn3Cre_P21_1_240,Ngn3-Cre; Rosa-RFP,-,-,-,P21,1,339528,198618,193372,44600,5202,alpha,-
Ngn3Cre_P21_1_241,Ngn3-Cre; Rosa-RFP,-,-,-,P21,1,243252,148923,130903,44237,5991,alpha,-


C: The xlsx table should have all necesary metadata

In [18]:
# Parse obs
obs=obs[['Strain', 'Treatment', 'Insulin implants', 'Time after STZ',
       'Age at death', 'Batch', 'Putative Cell Type', 'Putative β-cell Group']].copy()
obs.columns=['strain','STZ','insulin_implant','time_after_STZ','age','batch',
             'cell_type_original','cell_subtype_original']

In [19]:
obs['cell_type_original'].value_counts(dropna=False)

beta           1005
acinar          780
alpha           616
delta           255
PP              167
duct            131
immune           42
mesenchyme        2
endothelium       1
Name: cell_type_original, dtype: int64

C: All cells are annotated

In [24]:
# Parse obs values
obs['STZ']=obs['STZ'].apply(lambda x: True if x=='STZ' else False)
obs['insulin_implant']=obs['insulin_implant'].apply(lambda x: True if 'Yes' in x else False)
obs['time_after_STZ']=obs['time_after_STZ'].apply(
    lambda x: np.nan if x=='-' else x[1:]+' '+x[0].lower())
obs['age']=obs['age'].map({'2M':'2 m', '4M':'4 m','2M 12D':'2.4 m','2M 23D':'2.77 m','3M':'3 m',
                          '3M 12D':'3.4 m','2M 6D':'2.2 m','7M':'7 m','11M':'11 m',
                          'P3':'3 d','P12':'12 d','P21':'21 d'})
obs['cell_type']=obs['cell_type_original'].replace({
    'PP':'gamma','duct':'ductal','endothelium':'endothelial'})
obs['cell_subtype_original']=obs['cell_subtype_original'].apply(
    lambda x: np.nan if x=='-' else 'beta_'+str(x))
obs['donor']=pd.Series(obs.index).apply(lambda x: '_'.join(x.split('_')[:-1])).values
obs['sex']='male'
obs['disease']=obs['STZ'].map({True:'T2D',False:'healthy'})

In [25]:
obs

,strain,STZ,insulin_implant,time_after_STZ,age,batch,cell_type_original,cell_subtype_original,cell_type,donor,sex,disease
Sample,,,,,,,,,,,,
Ngn3Cre_Control_1_001,Ngn3-Cre; Rosa-RFP,False,False,NaN,2 m,1,acinar,NaN,acinar,Ngn3Cre_Control_1,male,healthy
Ngn3Cre_Control_1_002,Ngn3-Cre; Rosa-RFP,False,False,NaN,2 m,1,beta,beta_1,beta,Ngn3Cre_Control_1,male,healthy
Ngn3Cre_Control_1_003,Ngn3-Cre; Rosa-RFP,False,False,NaN,2 m,1,beta,beta_1,beta,Ngn3Cre_Control_1,male,healthy
Ngn3Cre_Control_1_004,Ngn3-Cre; Rosa-RFP,False,False,NaN,2 m,1,beta,beta_1,beta,Ngn3Cre_Control_1,male,healthy
Ngn3Cre_Control_1_005,Ngn3-Cre; Rosa-RFP,False,False,NaN,2 m,1,beta,beta_1,beta,Ngn3Cre_Control_1,male,healthy
...,...,...,...,...,...,...,...,...,...,...,...,...
Ngn3Cre_P21_1_239,Ngn3-Cre; Rosa-RFP,False,False,NaN,21 d,1,beta,beta_1,beta,Ngn3Cre_P21_1,male,healthy
Ngn3Cre_P21_1_240,Ngn3-Cre; Rosa-RFP,False,False,NaN,21 d,1,alpha,NaN,alpha,Ngn3Cre_P21_1,male,healthy
Ngn3Cre_P21_1_241,Ngn3-Cre; Rosa-RFP,False,False,NaN,21 d,1,alpha,NaN,alpha,Ngn3Cre_P21_1,male,healthy


In [26]:
obs.drop_duplicates('donor')

,strain,STZ,insulin_implant,time_after_STZ,age,batch,cell_type_original,cell_subtype_original,cell_type,donor,sex,disease
Sample,,,,,,,,,,,,
Ngn3Cre_Control_1_001,Ngn3-Cre; Rosa-RFP,False,False,NaN,2 m,1,acinar,NaN,acinar,Ngn3Cre_Control_1,male,healthy
Ngn3Cre_Control_2_001,Ngn3-Cre; Rosa-RFP,False,False,NaN,4 m,2,acinar,NaN,acinar,Ngn3Cre_Control_2,male,healthy
Ngn3Cre_Control_3_001,Ngn3-Cre; Rosa-RFP,False,False,NaN,4 m,3,beta,beta_1,beta,Ngn3Cre_Control_3,male,healthy
Ngn3Cre_STZ_D12_1_001,Ngn3-Cre; Rosa-RFP,True,True,12 d,2.4 m,1,acinar,NaN,acinar,Ngn3Cre_STZ_D12_1,male,T2D
Ngn3Cre_STZ_D12_2_001,Ngn3-Cre; Rosa-RFP,True,True,12 d,2.77 m,2,beta,beta_2,beta,Ngn3Cre_STZ_D12_2,male,T2D
Ngn3Cre_STZ_D30_1_001,Ngn3-Cre; Rosa-RFP,True,True,30 d,3 m,1,PP,NaN,gamma,Ngn3Cre_STZ_D30_1,male,T2D
Ngn3Cre_STZ_D42_1_001,Ngn3-Cre; Rosa-RFP,True,True,42 d,3.4 m,1,delta,NaN,delta,Ngn3Cre_STZ_D42_1,male,T2D
Ngn3Cre_STZ_D6_1_001,Ngn3-Cre; Rosa-RFP,True,True,6 d,2.2 m,1,acinar,NaN,acinar,Ngn3Cre_STZ_D6_1,male,T2D
Ngn3Cre_STZ_M2_1_001,Ngn3-Cre; Rosa-RFP,True,True,2 m,4 m,1,acinar,NaN,acinar,Ngn3Cre_STZ_M2_1,male,T2D


Adata

In [27]:
adata.obs=obs.reindex(adata.obs_names)

In [28]:
# Log normalise
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

In [29]:
adata

AnnData object with n_obs × n_vars = 2999 × 40824
    obs: 'strain', 'STZ', 'insulin_implant', 'time_after_STZ', 'age', 'batch', 'cell_type_original', 'cell_subtype_original', 'cell_type', 'donor', 'sex', 'disease'
    var: 'gene_symbol'
    uns: 'log1p'
    layers: 'raw', 'normalised_original'

In [30]:
for col in adata.obs.columns:
    print(col)
    print(adata.obs[col].unique().tolist())

strain
['Ngn3-Cre; Rosa-RFP']
STZ
[False, True]
insulin_implant
[False, True]
time_after_STZ
[nan, '12 d', '30 d', '42 d', '6 d', '2 m', '5 m', '9 m']
age
['2 m', '4 m', '2.4 m', '2.77 m', '3 m', '3.4 m', '2.2 m', '7 m', '11 m', '12 d', '21 d', '3 d']
batch
[1, 2, 3]
cell_type_original
['acinar', 'beta', 'delta', 'alpha', 'PP', 'duct', 'immune', 'endothelium', 'mesenchyme']
cell_subtype_original
[nan, 'beta_1', 'beta_2', 'beta_3']
cell_type
['acinar', 'beta', 'delta', 'alpha', 'gamma', 'ductal', 'immune', 'endothelial', 'mesenchyme']
donor
['Ngn3Cre_Control_1', 'Ngn3Cre_Control_2', 'Ngn3Cre_Control_3', 'Ngn3Cre_STZ_D12_1', 'Ngn3Cre_STZ_D12_2', 'Ngn3Cre_STZ_D30_1', 'Ngn3Cre_STZ_D42_1', 'Ngn3Cre_STZ_D6_1', 'Ngn3Cre_STZ_M2_1', 'Ngn3Cre_STZ_M2_NoIns_1', 'Ngn3Cre_STZ_M2_NoIns_2', 'Ngn3Cre_STZ_M5_1', 'Ngn3Cre_STZ_M9_1', 'Ngn3Cre_P12_1', 'Ngn3Cre_P12_2', 'Ngn3Cre_P21_1', 'Ngn3Cre_P3_1']
sex
['male']
disease
['healthy', 'T2D']


In [31]:
if SAVE:
    adata.write(path_ds+'adata.h5ad')

/home/icb/karin.hrovatin/miniconda3/envs/rpy2_3/lib/python3.8/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'strain' as categorical
/home/icb/karin.hrovatin/miniconda3/envs/rpy2_3/lib/python3.8/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'time_after_STZ' as categorical
/home/icb/karin.hrovatin/miniconda3/envs/rpy2_3/lib/python3.8/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.

Prepare per-sample scran normalisation

In [5]:
# Load data
adata=sc.read(path_ds+'adata.h5ad')

In [6]:
# Use raw
adata.X=adata.layers['raw']

In [12]:
for sample, idx_sample in adata.obs.groupby('donor').groups.items():
    # Subset data
    adata_sub=adata[idx_sample,:].copy()
    print('Normalising:',sample,adata_sub.shape)
    # Faster on sparse matrices
    if not sparse.issparse(adata_sub.X): 
        adata_sub.X = sparse.csr_matrix(adata_sub.X)
    # Sort indices is necesary for conversion to R object 
    adata_sub.X.sort_indices()
    
    # Prepare clusters for scran
    adata_sub_pp=adata_sub.copy()
    sc.pp.normalize_total(adata_sub_pp, target_sum=1e6, exclude_highly_expressed=True)
    sc.pp.log1p(adata_sub_pp)
    sc.pp.pca(adata_sub_pp, n_comps=15)
    sc.pp.neighbors(adata_sub_pp)
    sc.tl.louvain(adata_sub_pp, key_added='groups', resolution=1)
    
    # Normalise
    ro.globalenv['data_mat'] = adata_sub.X.T.todense()
    ro.globalenv['input_groups'] = adata_sub_pp.obs['groups']
    try:
        size_factors = ro.r(f'calculateSumFactors(data_mat, clusters = input_groups, min.mean = 0.1, BPPARAM=MulticoreParam(workers = 16))')
    except:
        # Sometimes the above does not work so change parameter
        size_factors = ro.r(f'calculateSumFactors(data_mat, clusters = input_groups, min.mean = 0.2, BPPARAM=MulticoreParam(workers = 16))')
    adata.obs.loc[adata_sub.obs.index,'size_factors_sample'] = size_factors

del adata_sub
del adata_sub_pp

Normalising: Ngn3Cre_Control_1 (295, 40824)
Normalising: Ngn3Cre_Control_2 (270, 40824)
Normalising: Ngn3Cre_Control_3 (22, 40824)
Normalising: Ngn3Cre_P3_1 (204, 40824)
Normalising: Ngn3Cre_P12_1 (78, 40824)
Normalising: Ngn3Cre_P12_2 (74, 40824)
Normalising: Ngn3Cre_P21_1 (243, 40824)
Normalising: Ngn3Cre_STZ_D6_1 (246, 40824)
Normalising: Ngn3Cre_STZ_D12_1 (218, 40824)
Normalising: Ngn3Cre_STZ_D12_2 (182, 40824)
Normalising: Ngn3Cre_STZ_D30_1 (277, 40824)
Normalising: Ngn3Cre_STZ_D42_1 (240, 40824)
Normalising: Ngn3Cre_STZ_M2_1 (181, 40824)
Normalising: Ngn3Cre_STZ_M2_NoIns_1 (76, 40824)
Normalising: Ngn3Cre_STZ_M2_NoIns_2 (51, 40824)
Normalising: Ngn3Cre_STZ_M5_1 (153, 40824)
Normalising: Ngn3Cre_STZ_M9_1 (189, 40824)


In [13]:
# Save parse anno and colors
if SAVE:
    h.update_adata(
            adata_new=adata, path=path_ds+'adata.h5ad',
            io_copy=False,
            add=[('obs',True,'size_factors_sample','size_factors_sample')],
        rm=None)

## GSE83146 - SMARTer

In [31]:
dataset='GSE83146'
path_ds=data_path+dataset+'/GEO/'

In [32]:
x=pd.read_table(path_ds+'GSE83146_expr_tpm.txt',index_col=0)

In [33]:
x

,Beta cell 1,Beta cell 2,Beta cell 3,Beta cell 4,Beta cell 5,Beta cell 6,Beta cell 7,Beta cell 8,Beta cell 9,Beta cell 10,...,Beta cell 198,Beta cell 199,Beta cell 200,Beta cell 201,Beta cell 202,Beta cell 203,Beta cell 204,Beta cell 205,Beta cell 206,Beta cell 207
gene.id,,,,,,,,,,,,,,,,,,,,,
11287,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
11298,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
11302,0.000000,0.000000,1.765902,0.000000,2.892076,0.000000,0.000000,0.000000,0.0,0.000000,...,1.610063,0.000000,0.00000,0.547936,0.000000,2.454366,0.0,0.000000,0.000000,0.0
11303,0.052266,7.724282,0.000000,0.168357,13.330814,3.104047,45.806835,0.073593,0.0,5.638172,...,20.678926,3.439675,96.42474,3.661840,0.060868,0.703500,0.0,0.071231,26.527109,0.0
11304,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102640996,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
102640997,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
102640998,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0


C: The gene ids are entrez ids

In [34]:
mg = mygene.MyGeneInfo()
genemap = mg.querymany(x.index.to_list(), scopes='entrezgene', 
                       fields=['ensembl.gene','symbol'],  species='mouse')

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-20000...done.
querying 20001-21000...done.
querying 21001-22000...done.
querying 22001-23000...done.
querying 23001-24000...done.
querying 24001-25000...done.
querying 25001-26000...done.
querying 26001-27000...done.
querying 27001-28000...done.
querying 28001-29000...done.
querying 29001-30000...done.
querying 30001-31000...done.
querying 31001-32000...done.
querying 32001-33000...done.
querying 33001-34000...done.
querying 34001-35000...done.
queryin

In [35]:
genemap_df=[]
for g in genemap:
    g_parsed={'uid':g['query']}
    g_parsed['gene_symbol']=g['symbol'] if 'symbol' in g else np.nan
    # Genes with multiple EIDs have these as list
    if 'ensembl' in g:
        if isinstance(g['ensembl'],list):
            g_parsed['EID']=','.join([gs['gene'] for gs in g['ensembl']])
        else:
            g_parsed['EID']=g['ensembl']['gene']
    genemap_df.append(g_parsed)
genemap_df=pd.DataFrame(genemap_df)
genemap_df.index=genemap_df.uid
genemap_df.drop('uid',axis=1,inplace=True)

In [36]:
genemap_df

,gene_symbol,EID
uid,,
11287,Pzp,ENSMUSG00000030359
11298,Aanat,ENSMUSG00000020804
11302,Aatk,ENSMUSG00000025375
11303,Abca1,ENSMUSG00000015243
11304,Abca4,ENSMUSG00000028125
...,...,...
102640996,Gm36922,NaN
102640997,NaN,NaN
102640998,Gm36924,NaN


In [37]:
adata=sc.AnnData(X=csr_matrix(x.T),obs=pd.DataFrame(index=x.columns),
                 var=pd.DataFrame(index=x.index))

/home/icb/karin.hrovatin/miniconda3/envs/rpy2_3/lib/python3.8/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [38]:
# Logtransform
adata.layers['normalised_original']=adata.X.copy()
sc.pp.log1p(adata)

In [39]:
# Add gene info
for col in genemap_df:
    adata.var[col]=genemap_df[col]

In [40]:
obs=pd.read_table(path_ds+'GSE83146_series_matrix.txt',index_col=0,skiprows=26)

In [41]:
obs

,Beta cell 1,Beta cell 2,Beta cell 3,Beta cell 4,Beta cell 5,Beta cell 6,Beta cell 7,Beta cell 8,Beta cell 9,Beta cell 10,...,Beta cell 198,Beta cell 199,Beta cell 200,Beta cell 201,Beta cell 202,Beta cell 203,Beta cell 204,Beta cell 205,Beta cell 206,Beta cell 207
!Sample_title,,,,,,,,,,,,,,,,,,,,,
!Sample_geo_accession,GSM2195203,GSM2195204,GSM2195205,GSM2195206,GSM2195207,GSM2195208,GSM2195209,GSM2195210,GSM2195211,GSM2195212,...,GSM2195400,GSM2195401,GSM2195402,GSM2195403,GSM2195404,GSM2195405,GSM2195406,GSM2195407,GSM2195408,GSM2195409
!Sample_status,Public on Jun 30 2016,Public on Jun 30 2016,Public on Jun 30 2016,Public on Jun 30 2016,Public on Jun 30 2016,Public on Jun 30 2016,Public on Jun 30 2016,Public on Jun 30 2016,Public on Jun 30 2016,Public on Jun 30 2016,...,Public on Jun 30 2016,Public on Jun 30 2016,Public on Jun 30 2016,Public on Jun 30 2016,Public on Jun 30 2016,Public on Jun 30 2016,Public on Jun 30 2016,Public on Jun 30 2016,Public on Jun 30 2016,Public on Jun 30 2016
!Sample_submission_date,Jun 09 2016,Jun 09 2016,Jun 09 2016,Jun 09 2016,Jun 09 2016,Jun 09 2016,Jun 09 2016,Jun 09 2016,Jun 09 2016,Jun 09 2016,...,Jun 09 2016,Jun 09 2016,Jun 09 2016,Jun 09 2016,Jun 09 2016,Jun 09 2016,Jun 09 2016,Jun 09 2016,Jun 09 2016,Jun 09 2016
!Sample_last_update_date,May 15 2019,May 15 2019,May 15 2019,May 15 2019,May 15 2019,May 15 2019,May 15 2019,May 15 2019,May 15 2019,May 15 2019,...,May 15 2019,May 15 2019,May 15 2019,May 15 2019,May 15 2019,May 15 2019,May 15 2019,May 15 2019,May 15 2019,May 15 2019
!Sample_type,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA,...,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA
!Sample_channel_count,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
!Sample_source_name_ch1,Pancreatic islets,Pancreatic islets,Pancreatic islets,Pancreatic islets,Pancreatic islets,Pancreatic islets,Pancreatic islets,Pancreatic islets,Pancreatic islets,Pancreatic islets,...,Pancreatic islets,Pancreatic islets,Pancreatic islets,Pancreatic islets,Pancreatic islets,Pancreatic islets,Pancreatic islets,Pancreatic islets,Pancreatic islets,Pancreatic islets
!Sample_organism_ch1,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,...,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus,Mus musculus
!Sample_characteristics_ch1,tissue: Pancreatic islets,tissue: Pancreatic islets,tissue: Pancreatic islets,tissue: Pancreatic islets,tissue: Pancreatic islets,tissue: Pancreatic islets,tissue: Pancreatic islets,tissue: Pancreatic islets,tissue: Pancreatic islets,tissue: Pancreatic islets,...,tissue: Pancreatic islets,tissue: Pancreatic islets,tissue: Pancreatic islets,tissue: Pancreatic islets,tissue: Pancreatic islets,tissue: Pancreatic islets,tissue: Pancreatic islets,tissue: Pancreatic islets,tissue: Pancreatic islets,tissue: Pancreatic islets


In [42]:
# Subset obs
obs=obs.T.iloc[:,[0,10,11]]

In [43]:
obs.columns=['geo_accession','age','sex']
obs['age']=obs['age'].apply(lambda x: x.split(': ')[1].replace('months','m'))
obs['sex']=obs['sex'].str.replace('Sex: ','')
obs['cell_type']='beta'
obs['disease']='healthy'

C: All cells were annotated as beta

In [44]:
obs

,geo_accession,age,sex,cell_type,disease
Beta cell 1,GSM2195203,3 m,male,beta,healthy
Beta cell 2,GSM2195204,3 m,male,beta,healthy
Beta cell 3,GSM2195205,3 m,male,beta,healthy
Beta cell 4,GSM2195206,3 m,male,beta,healthy
Beta cell 5,GSM2195207,3 m,male,beta,healthy
...,...,...,...,...,...
Beta cell 203,GSM2195405,26 m,male,beta,healthy
Beta cell 204,GSM2195406,26 m,male,beta,healthy
Beta cell 205,GSM2195407,26 m,male,beta,healthy
Beta cell 206,GSM2195408,26 m,male,beta,healthy


In [45]:
adata.obs=obs.reindex(adata.obs_names)

In [46]:
adata

AnnData object with n_obs × n_vars = 207 × 44448
    obs: 'geo_accession', 'age', 'sex', 'cell_type', 'disease'
    var: 'gene_symbol', 'EID'
    uns: 'log1p'
    layers: 'normalised_original'

In [47]:
for col in adata.obs.columns:
    if col!='geo_accession':
        print(col)
        print(adata.obs[col].unique().tolist())

age
['3 m', '26 m']
sex
['male']
cell_type
['beta']
disease
['healthy']


In [48]:
if SAVE:
    adata.write(path_ds+'adata.h5ad')

/home/icb/karin.hrovatin/miniconda3/envs/rpy2_3/lib/python3.8/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'age' as categorical
/home/icb/karin.hrovatin/miniconda3/envs/rpy2_3/lib/python3.8/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'sex' as categorical
/home/icb/karin.hrovatin/miniconda3/envs/rpy2_3/lib/python3.8/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_catego